# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, Sst, Male, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISl sessions. The Female SST VISp sessions have 0 low-contamination units. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('Sst') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISl' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
756029989,2019-10-03T00:00:00Z,734865738,brain_observatory_1.1,96.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,684,2214,6,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."


In [7]:
session_id = filtered_sessions.index.values[0]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 884


,snr,presence_ratio,waveform_spread,cumulative_drift,isolation_distance,waveform_amplitude,firing_rate,nn_hit_rate,isi_violations,waveform_velocity_below,L_ratio,nn_miss_rate,waveform_halfwidth,waveform_repolarization_slope,amplitude_cutoff,waveform_PT_ratio,silhouette_score,waveform_recovery_slope,waveform_duration,local_index_unit,max_drift,waveform_velocity_above,cluster_id,peak_channel_id,d_prime,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950910352,2.816993,0.99,60.0,419.57,69.455405,106.785900,4.532385,0.935531,0.029797,0.000000,0.002020,0.008277,0.096147,0.365539,0.057700,0.476196,0.080869,-0.153573,0.151089,6,34.38,-0.137353,6,850264156,4.576155,NaN,200.0,8.573516,NaN,3.563571,3.013333,3.472371,0.506292,0.121946,0.092779,0.109856,25.000,1.312811,0.479,0.32,4.0,0.987385,NaN,0.970335,1.000000,0.706667,180.0,120.0,0.000020,NaN,3.094523e-07,0.010614,3.288081e-03,50.000,4908,0.50,3.679539,0.0,5.399795,2.173257,4.740704,NaN,0.0185,False,False,False,False,False,0.074966,False,0.124258,NaN,0.053529,0.244172,0.055879,60,11,810755797,5,215.0,APN,8157.0,3521.0,6697.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910364,2.592082,0.99,60.0,231.42,102.847616,85.808775,31.486161,0.995333,0.005998,0.343384,0.000146,0.002786,0.274707,0.147357,0.065649,0.953652,0.153496,-0.010004,0.480737,7,23.43,-0.618090,7,850264158,5.602703,NaN,800.0,0.696467,NaN,1.127965,1.382684,1.474619,0.553213,0.037434,0.029263,0.032803,51.250,2.509759,0.979,0.16,4.0,0.148090,NaN,-0.017989,0.172414,0.015326,225.0,120.0,0.001521,NaN,8.555631e-01,0.236355,9.013584e-01,-18.750,4951,0.50,26.488752,0.0,31.565193,27.097733,31.374439,NaN,0.1475,False,False,False,False,False,0.030797,False,0.013569,NaN,0.003357,0.024370,0.004924,80,59,810755797,6,215.0,APN,8154.0,3513.0,6698.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910371,2.095077,0.99,70.0,209.31,76.907610,76.898055,17.328610,0.993333,0.005242,0.137353,0.004522,0.007975,0.164824,0.299231,0.015509,0.615593,0.089229,-0.071484,0.288442,8,57.44,0.274707,8,850264164,5.061817,NaN,300.0,6.767485,NaN,1.235328,3.876923,1.301639,0.223564,0.048717,0.050593,0.018864,86.667,1.024316,0.497,0.04,4.0,0.213891,NaN,0.204641,0.511111,-0.015873,225.0,30.0,0.329712,NaN,9.284294e-02,0.161964,9.037874e-01,46.667,4942,0.75,14.373947,0.0,17.881994,12.173311,19.064063,NaN,0.0435,False,False,False,False,False,0.005763,False,0.022789,NaN,0.005609,0.050257,0.005125,100,11,810755797,9,215.0,APN,8146.0,3487.0,6701.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910392,3.024744,0.99,70.0,179.71,65.671206,120.519555,16.262414,0.944000,0.096884,-0.274707,0.007306,0.002874,0.109883,0.497249,0.025891,0.616317,0.139601,-0.116365,0.206030,11,33.65,0.000000,11,850264172,4.219074,NaN,100.0,14.891282,NaN,6.331037,4.973913,5.240884,0.286195,0.089741,0.038626,0.066273,10.000,0.915356,0.942,0.08,4.0,1.000000,NaN,0.937998,0.760504,0.791262,180.0,150.0,0.000002,NaN,2.323203e-11,0.000867

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISl') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 32


,snr,presence_ratio,waveform_spread,cumulative_drift,isolation_distance,waveform_amplitude,firing_rate,nn_hit_rate,isi_violations,waveform_velocity_below,L_ratio,nn_miss_rate,waveform_halfwidth,waveform_repolarization_slope,amplitude_cutoff,waveform_PT_ratio,silhouette_score,waveform_recovery_slope,waveform_duration,local_index_unit,max_drift,waveform_velocity_above,cluster_id,peak_channel_id,d_prime,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950939257,2.338750,0.99,40.0,78.73,88.970340,112.858785,18.380085,0.973333,0.060630,0.000000,0.010463,0.004667,0.068677,0.411564,0.003191,0.530065,0.070160,-0.060808,0.302178,268,32.22,0.000000,271,850260417,3.686532,NaN,1200.0,9.937133,1.157146,2.187302,4.644961,2.822991,0.287805,0.026995,0.016491,0.038250,81.667,3.319669,0.297,0.08,2.0,0.618170,0.655289,0.414629,0.758621,0.409326,315.0,30.0,0.000065,1.005357e-12,0.000002,0.108401,0.001284,14.167,4998,0.75,12.214137,15.720257,25.446007,5.010992,23.939987,NaN,0.0355,False,False,False,False,False,0.320525,False,0.039735,0.047475,0.072172,0.174711,0.031762,2480,27,810755803,247,409.0,VISl,8666.0,2312.0,9400.0,probeD,See electrode locations,29999.9081,1249.996171,True
950939347,2.077726,0.99,60.0,153.07,78.215101,165.387495,2.615783,0.997455,0.056908,0.618090,0.000269,0.000068,2.417420,0.056352,0.006627,1.445413,0.166401,-0.130314,1.222446,273,40.76,0.686767,276,850260425,5.335018,NaN,600.0,9.874510,1.442051,2.734627,0.716667,3.317718,1.125807,0.325905,0.512464,0.285441,15.000,0.224601,0.096,0.02,1.0,0.986667,1.000000,0.915517,0.314286,0.876923,0.0,90.0,0.038119,3.620308e-03,0.000018,0.564192,0.000035,40.000,4951,0.00,0.271968,0.559535,2.347591,0.612942,1.705241,NaN,0.1145,False,False,False,False,False,0.342898,False,0.722023,0.107296,0.093997,0.344324,0.278817,2520,27,810755803,251,409.0,VISl,8686.0,2278.0,9428.0,probeD,See electrode locations,29999.9081,1249.996171,True
950942974,3.390476,0.99,50.0,89.61,90.490973,206.421735,3.215110,0.985360,0.003767,0.000000,0.000970,0.000790,0.178559,0.682083,0.000943,0.157660,0.157839,-0.093396,0.659296,465,38.93,0.686767,472,850260423,4.988579,NaN,500.0,6.355257,0.449167,0.565155,0.844444,0.329670,0.595401,0.157750,0.292910,0.035214,90.000,0.387280,0.006,0.02,8.0,0.473389,0.549261,0.236364,0.250000,0.263323,180.0,0.0,0.153492,2.001802e-03,0.082396,0.696817,0.014305,20.000,4917,0.25,2.759522,2.850962,4.557484,0.839917,3.555694,NaN,0.1255,False,False,False,False,False,0.084712,False,0.274488,0.773803,0.069111,0.286792,0.099358,2520,59,810755803,250,409.0,VISl,8682.0,2285.0,9423.0,probeD,See electrode locations,29999.9081,1249.996171,True
950939479,1.537521,0.99,50.0,64.13,77.644767,221.672295,10.258889,0.984667,0.014799,-0.480737,0.009355,0.002866,0.370854,0.668138,0.006977,0.221193,0.004660,-0.054660,0.782915,281,40.56,0.686767,284,850260431,3.781295,NaN,900.0,11.466667,1.556344,1.340331,1.176923,4.133333,0.742672,0.098099,0.154322,0.031945,36.667,0.598307,0.000,0.32,15.0,-0.681818,-

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
950939257    23.939987
950939347     1.705241
950942974     3.555694
950939479     4.379672
950939509     5.476089
950939641     1.906406
950940157     0.103913
950940001     0.453621
950939945     7.395151
950940104     6.618467
950940121     4.292412
950940040     4.869263
950940023     7.228624
950940237    15.235264
950942155    11.772825
950940185     4.849279
950940171     8.764007
950940311     1.683260
950940453     3.297910
950940437     0.663445
950940545     1.962360
950940526     2.116897
950940507     2.910900
950940615    16.838590
950942252     0.211823
950942235     5.558686
950940631     1.335550
950940688     4.488248
950940671     1.196333
950940718     2.164857
950942304     0.449624
950940859     1.601328
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

4.969553951881359


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
950939257    12.214137
950939347     0.271968
950942974     2.759522
950939479    14.154036
950939509     5.113747
950939641     0.748975
950940157     0.116861
950940001     1.035817
950939945     5.109497
950940104     4.307935
950940121     4.843372
950940040    10.008113
950940023     8.248287
950940237    19.219444
950942155     9.012667
950940185     6.894289
950940171     8.271128
950940311     1.843754
950940453     5.472830
950940437     0.278343
950940545     2.093943
950940526     1.479359
950940507     3.230154
950940615     7.813776
950942252     0.152451
950942235     3.030959
950940631     0.404234
950940688     4.189480
950940671     0.465852
950940718     1.307254
950942304     0.296403
950940859     1.890498
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

4.571221368054704


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])